<a href="https://colab.research.google.com/github/fatemehrashidi7/Shabake_Mokhaberati/blob/main/crc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

# ==============================================================================
# ۱. توابع اصلی CRC (بدون تغییر نسبت به قبل)
# ==============================================================================

def xor_operation(a, b):
    result = ['0' if a[i] == b[i] else '1' for i in range(len(a))]
    return ''.join(result)

def binary_division(dividend, divisor):
    n = len(divisor)
    current_remainder = dividend[0:n]

    for i in range(n, len(dividend) + 1):
        if current_remainder[0] == '1':
            current_remainder = xor_operation(current_remainder, divisor)

        if i < len(dividend):
            current_remainder = current_remainder[1:] + dividend[i]
        elif len(current_remainder) > 1:
             current_remainder = current_remainder[1:]

    return current_remainder

def generate_crc(data_bits, generator_poly):
    n = len(generator_poly) - 1
    padded_data = data_bits + '0' * n
    crc = binary_division(padded_data, generator_poly)
    codeword = data_bits + crc
    return crc, codeword

def check_crc(received_codeword, generator_poly):
    remainder = binary_division(received_codeword, generator_poly)
    is_valid = ('1' not in remainder)
    return is_valid, remainder

def inject_error(codeword, position):
    codeword_list = list(codeword)
    codeword_list[position] = '1' if codeword_list[position] == '0' else '0'
    return ''.join(codeword_list)

# ==============================================================================
# ۲. اجرای شبیه‌سازی و تولید خروجی در قالب DataFrame
# ==============================================================================

def run_crc_simulation(data_bits, generator_poly, name):
    """اجرای شبیه‌سازی برای یک مدل CRC خاص و بازگرداندن نتایج"""

    # --- فاز ۱: ارسال صحیح ---
    crc_generated, codeword_clean = generate_crc(data_bits, generator_poly)

    # --- فاز ۲: دریافت صحیح (بدون خطا) ---
    is_valid_clean, rem_clean = check_crc(codeword_clean, generator_poly)

    # --- فاز ۳: دریافت با خطا (تزریق خطا در موقعیت دلخواه) ---
    error_pos = len(data_bits) + 5 # خطا را در ۵مین بیت بعد از داده اصلی قرار می‌دهیم
    if error_pos >= len(codeword_clean):
         error_pos = len(data_bits)

    codeword_error = inject_error(codeword_clean, error_pos)
    is_valid_error, rem_error = check_crc(codeword_error, generator_poly)

    results = [
        # حالت ۱: بدون خطا
        {
            'مدل CRC': name,
            'حالت': 'دریافت صحیح (بدون خطا)',
            'پلی‌نومیال': generator_poly,
            'داده ورودی (پیام)': data_bits,
            'CRC تولید شده': crc_generated,
            'پیام دریافتی': codeword_clean,
            'باقیمانده چک': rem_clean,
            'نتیجه تشخیص': '✅ صحیح (Passed)' if is_valid_clean else '❌ خطا (Failed)'
        },
        # حالت ۲: با خطا
        {
            'مدل CRC': name,
            'حالت': f'دریافت با خطا (موقعیت {error_pos})',
            'پلی‌نومیال': generator_poly,
            'داده ورودی (پیام)': data_bits,
            'CRC تولید شده': crc_generated,
            'پیام دریافتی': codeword_error,
            'باقیمانده چک': rem_error,
            'نتیجه تشخیص': '✅ صحیح (Passed)' if is_valid_error else '❌ خطا (Failed)'
        }
    ]
    return results

# ==============================================================================
# ۳. اجرای مدل‌های مختلف و جمع‌آوری نتایج
# ==============================================================================

# پلی‌نومیال‌ها:
# CRC-8 (x^8 + x^7 + x^2 + x + 1) -> برای تقسیم باید یک بیت MSB حذف شود
CRC8_POLY = '110000111'
# CRC-16-CCITT (x^16 + x^12 + x^5 + 1) -> برای تقسیم
CRC16_POLY = '10001000000100001'

DATA_A = '101100101' # داده نمونه ۱
DATA_B = '1101011001101010' # داده نمونه ۲

all_results = []

# اجرای CRC-8
results_8 = run_crc_simulation(DATA_A, CRC8_POLY, 'CRC-8')
all_results.extend(results_8)

# اجرای CRC-16
results_16 = run_crc_simulation(DATA_B, CRC16_POLY, 'CRC-16-CCITT')
all_results.extend(results_16)

# تبدیل نتایج به DataFrame و نمایش
results_df = pd.DataFrame(all_results)

# نمایش خروجی با فرمت جدولی
print("## 📊 نتایج شبیه‌سازی CRC برای مدل‌های مختلف (CRC-8 و CRC-16) ##\n")
# استفاده از display برای نمایش بهتر در محیط Jupyter/Colab
display(results_df)

# خلاصه وضعیت
print("\n" + "="*80)
print(f"خلاصه نتایج:")
print(f"- در حالت 'دریافت صحیح'، باقیمانده (Rem.) باید صفر باشد. (مشاهده: {results_df['باقیمانده چک'].iloc[0]})")
print(f"- در حالت 'دریافت با خطا'، سیستم باقیمانده‌ای غیر صفر محاسبه کرده و خطا را تشخیص می‌دهد.")
print("="*80)

## 📊 نتایج شبیه‌سازی CRC برای مدل‌های مختلف (CRC-8 و CRC-16) ##



,مدل CRC,حالت,پلی‌نومیال,داده ورودی (پیام),CRC تولید شده,پیام دریافتی,باقیمانده چک,نتیجه تشخیص
0,CRC-8,دریافت صحیح (بدون خطا),110000111,101100101,00111010,10110010100111010,00000000,✅ صحیح (Passed)
1,CRC-8,دریافت با خطا (موقعیت 14),110000111,101100101,00111010,10110010100111110,00000100,❌ خطا (Failed)
2,CRC-16-CCITT,دریافت صحیح (بدون خطا),10001000000100001,1101011001101010,0111001001101101,11010110011010100111001001101101,0000000000000000,✅ صحیح (Passed)
3,CRC-16-CCITT,دریافت با خطا (موقعیت 21),10001000000100001,1101011001101010,0111001001101101,11010110011010100111011001101101,0000010000000000,❌ خطا (Failed)



خلاصه نتایج:
- در حالت 'دریافت صحیح'، باقیمانده (Rem.) باید صفر باشد. (مشاهده: 00000000)
- در حالت 'دریافت با خطا'، سیستم باقیمانده‌ای غیر صفر محاسبه کرده و خطا را تشخیص می‌دهد.
